In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import expm
from scipy.special import comb
from scipy.optimize import curve_fit
import scipy.integrate as integrate
from scipy.interpolate import interp1d

from qutip import basis, fock, tensor
from qutip import rx, ry, rz, sigmax, sigmay, sigmaz, rotation
from qutip import destroy, create, num, cnot, qeye
from qutip import expect, thermal_dm, ket2dm
from qutip import Qobj, QobjEvo, sesolve

from tqdm.notebook import tqdm

from copy import deepcopy

import warnings
warnings.filterwarnings('ignore')

In [8]:
class State():
    
    """
    Class that contains most information and operations that you will need to simulate a LS gate
    takes as input a list of 4 or 5 parameters and optionally an int.
    
    :psi_init: a list of 4 or 5 parameters. The first 4 are the coefficients a, b, c, d, not necessarily
    normalized, of the input state a|gg>+b|ge>+c|eg>+d|ee>. The (optional) 5th is the motional occupation 
    number of the ions.
    :dim: dimension of the Fock state. Default is 20.
    
    """
    
    def __init__(self, psi_init, dim=None):
        
        # Initializes the state
        
        if dim==None:
            self.dim=20
        else:
            self.dim=dim
                
        if isinstance(psi_init, list) and len(psi_init) in [4, 5]:
            
            if len(psi_init)==5:
                self.n=psi_init[4]
            else:
                self.n=0
            
            psi_init = (tensor(basis(2,0), basis(2,0))*psi_init[0]+
                        tensor(basis(2,0), basis(2,1))*psi_init[1]+
                        tensor(basis(2,1), basis(2,0))*psi_init[2]+
                        tensor(basis(2,1), basis(2,1))*psi_init[3]).unit()
            
            self.psi_init = ket2dm(tensor(psi_init, fock(self.dim, self.n)))
            self.psi = ket2dm(tensor(psi_init, fock(self.dim, self.n)))
        
        else:
            self.psi_init = ket2dm(Qobj(psi_init))
            self.psi = ket2dm(Qobj(psi_init))
        
        if psi_init.shape[0]!=4:
            self.dim = psi_init.shape[0]//4
        
        # Initializes some constant properties and operators
    
        self.tg=np.pi
        self.eta=0.01
        
        self.n_loops=2
        
        self.alpha=0 # unequal illumination factor
        
        self.a = destroy(self.dim)
        self.ad = create(self.dim)
        
        self.X = (self.a+self.ad)/np.sqrt(2)
        self.P = 1j*(self.ad-self.a)/np.sqrt(2)
        
        self.times=np.linspace(0, self.tg, 101)
        
        self.fraction=1/5
        

    def reset_state(self):
        
        self.psi=deepcopy(self.psi_init)
        
    def reset_vars(self):
        
        self.tg=np.pi
        self.eta=0.01
        
        self.n_loops=1
        
        self.alpha=0
        
        self.a = destroy(self.dim)
        self.ad = create(self.dim)
        
        self.X = (self.a+self.ad)/np.sqrt(2)
        self.P = 1j*(self.ad-self.a)/np.sqrt(2)
        
    def pop(self, unitary=None):
        
        """
        :unitary: bool, True if working with a dm, False if with a ket
        finds the population of qubits in gg, ee, and ge/eg
        """
        
        if unitary==None:
            unitary=True
        
        if unitary:
            gg=self.psi[0,0]
            ee=self.psi[3*self.dim, 3*self.dim]
        else:
            gg=np.sum(self.psi[:self.dim]*np.conj(self.psi[:self.dim]))
            ee=np.sum(self.psi[3*self.dim:]*np.conj(self.psi[3*self.dim:]))
        
        return np.array([gg, ee, 1-gg-ee])
    
    def evol(self, t, args):
        # Time-dependent part of the Hamiltonian
        return np.exp(1j*(args['epsilon']*t+args['phi']))
    def evol_c(self, t, args):
        return np.exp(-1j*(args['epsilon']*t+args['phi']))
    
    def Sz(self):
        # As defined in Christopher Ballance's thesis
        Sz = 0.5*((self.alpha+1)*tensor(sigmaz(), qeye(2))+(self.alpha-1)*tensor(qeye(2), sigmaz()))
        return Sz
    
    def R2(self, phi=None):
        # Rotation by pi/2 with a phase phi
        if phi==None:
            phi=0
        r2 = rotation(sigmax()*np.cos(phi)+sigmay()*np.sin(phi), np.pi/2)
        r2=tensor(r2,r2,qeye(self.dim))
        self.psi = r2*self.psi*r2.dag()
        return r2

    def nR2(self, phi=None):
        # Rotation by -pi/2 with a phase phi
        if phi==None:
            phi=0
        r2 = rotation(sigmax()*np.cos(phi)+sigmay()*np.sin(phi), -np.pi/2)
        r2=tensor(r2,r2,qeye(self.dim))
        self.psi = r2*self.psi*r2.dag()
        return r2
            
    def unitary(self, time, phi, Sb_detuning=None, Rabi_detuning=None, psi_init=None):
        
        """
        Finds the evolution unitary for the LS gate, as given in Chris's thesis. Params similar to evolve().
        
        """
        
        if psi_init==None:
            psi_init=self.psi
    
        if Sb_detuning is None:
            Sb_detuning=1
        if Rabi_detuning is None:
            Rabi_detuning=1
            
        
        epsilon=2*np.pi/self.tg * Sb_detuning
        Omega=np.pi/self.tg / self.eta / np.sqrt(self.n_loops) * Rabi_detuning
        
        # For Sb_detuning=1 and Rabi_detuning=1, these return the values that
        # will close a loop in phase space and generate a phase of pi/4 for each loop
            
        disp = self.eta*Omega/epsilon * np.sin(epsilon*time/2) * np.exp(1j*(phi-epsilon*time/2))
        Phi = (self.eta*Omega/epsilon)**2 * (epsilon*time - np.sin(epsilon*time))
        
        unitary = self.displacement(disp*self.Sz())*(1j*Phi*tensor(self.Sz()*self.Sz(), 
                                                                             qeye(self.dim))).expm()
        return unitary
        
    def displacement(self, op):
        # Called by the unitary
        return (tensor(op,self.ad)-tensor(op.dag(),self.a)).expm()

    
    def detune(self, g_number, alpha=None, Sb=None, Rabi=None, factor_SPAM=None, factor_dep=None):
        
        """
        Finds the qubit populations for each value in a detuning range
        :factor: float between 0 and 1, represents the depolarizing parameter
        :g_number: int, number of gates applied
        :alpha: float between 0 and 1, represents the difference in illumination between
        the two ions (Rabi1-Rabi2)/(Rabi1+Rabi2)
        """
        
        if Sb is None:
            Sb=[1]
        if Rabi is None:
            Rabi=[1]
        if alpha is None:
            alpha=[0]
            
        pops=np.zeros(len(alpha)*len(Sb)*len(Rabi)*3).reshape((len(alpha), len(Sb), len(Rabi), 3))
        
        
        for step in tqdm(range(int(pops.size/3))):
            
            i=step//(len(Sb)*len(Rabi))
            j=(step%(len(Sb)*len(Rabi)))//len(Rabi)
            k=(step%(len(Sb)*len(Rabi)))%len(Rabi)
            
            self.alpha=alpha[i]
            self.reset_state()
            self.gate(g_number, Sb_detuning=Sb[j], Rabi_detuning=Rabi[k], factor_SPAM=factor_SPAM, factor_dep=factor_dep)
            pops[i,j,k]=self.pop()
        
        self.reset_vars()
        self.reset_state()
        
        return np.squeeze(pops)


    def gate(self, g_number=None, Sb_detuning=None, Rabi_detuning=None, factor_SPAM=None, factor_dep=None):
        """
        Applies a pi/2 pulse, sqrt(LS gate), pi pulse, sqrt(LS gate), pi/2 pulse
        """
        if g_number==None:
            g_number=1
        
        self.SPAM(factor_SPAM)
        
        self.R2(0)
        
        for g in range(g_number):
            self.psi = self.unitary(self.tg, np.pi*g, Sb_detuning, Rabi_detuning)*self.psi*self.unitary(self.tg, np.pi*g, Sb_detuning, Rabi_detuning).dag()
            self.R2(0)
            self.R2(0)
            self.psi = self.unitary(self.tg, np.pi*(g+1), Sb_detuning, Rabi_detuning)*self.psi*self.unitary(self.tg, np.pi*(g+1), Sb_detuning, Rabi_detuning).dag()
            self.R2(0)
            self.R2(0)
            self.depolarize(factor_dep)
        
        if g_number%2:
            self.R2(0)
        else:
            self.nR2(0)
            
        self.SPAM(factor_SPAM)

    
    def SPAM(self, factor):
        
        if factor==None:
            factor=0
        self.psi=(1-factor)*self.psi+factor*tensor(sigmax(), qeye(2), qeye(self.dim))*self.psi*tensor(sigmax(), qeye(2), qeye(self.dim))
        self.psi=(1-factor)*self.psi+factor*tensor(qeye(2), sigmax(), qeye(self.dim))*self.psi*tensor(qeye(2), sigmax(), qeye(self.dim))
        
        
    def depolarize(self, factor):
        
        if factor==None:
            factor=0
        self.psi=(1-factor)*self.psi+factor/3*(tensor(sigmax(), qeye(2), qeye(self.dim))*self.psi*tensor(sigmax(), qeye(2), qeye(self.dim))+
                                               tensor(sigmay(), qeye(2), qeye(self.dim))*self.psi*tensor(sigmay(), qeye(2), qeye(self.dim))+
                                               tensor(sigmaz(), qeye(2), qeye(self.dim))*self.psi*tensor(sigmaz(), qeye(2), qeye(self.dim)))
    
                                      
        self.psi=(1-factor)*self.psi+factor/3*(tensor(qeye(2), sigmax(), qeye(self.dim))*self.psi*tensor(qeye(2), sigmax(), qeye(self.dim))+
                                               tensor(qeye(2), sigmay(), qeye(self.dim))*self.psi*tensor(qeye(2), sigmay(), qeye(self.dim))+
                                               tensor(qeye(2), sigmaz(), qeye(self.dim))*self.psi*tensor(qeye(2), sigmaz(), qeye(self.dim)))